In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json
import time

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/lucy/processed/metadata')
db_path=os.path.join(root_dir,'db/lucy.db')

In [3]:
#define input tables and metadata
tbl_names_lst=["b_sample","l_sample"]
syn_tbl_name_lst=['b_sample_syn_01','l_sample_syn_01']

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

#connect to db
conn = sqlite3.connect(db_path)



In [4]:
query_obj=RND_QUERY(conn, tbl_names_lst, metadata_lst)

Metadata for table b_sample is valid.
Metadata for table l_sample is valid.
Parameter input is valid.


In [5]:
def gen_aggfltr_queries(query_obj: object,syn_tbl_name_lst: str) -> list:
    n_queries=10
    queries = []
    timeout=5 #maximum time in secs to execute the query, otherwise it will be skipped to next random query 
    for k in range(n_queries):
        query_obj.no_groupby_vars=2
        query_obj.no_where_vars=3
        rnd_query=query_obj.make_twin_aggfltr_query(syn_tbl_name_lst)
        matched_query=query_obj._match_twin_query(rnd_query)
        scored_query=query_obj.calc_dist_scores(matched_query)
        queries.append(scored_query)
        print('Generated Random Aggregate Filter Query - {} '.format(str(k)))
    return queries 

In [ ]:
gen_aggfltr_queries(query_obj,syn_tbl_name_lst)